# Using Classic Supervised ML Methods on Dataset
- The purpose of this experiment was to recreate some previous work on the dataset.
- What is the accuracy of some supervised ML methods on the data?
- We will first try using only a subset of genes as features that were intially provided to us by Dr. Raga Krishnakumar, and were derived in previous experiments.
- Much of this code was adapted from Dr. Raga Krishnakumar.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gseapy
!pip install scanpy
!pip install scikit-misc
!pip install igraph
!pip install pymc3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 76.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of pymc3 to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 78.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.5/869.5 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 12.1 MB/s eta 0:00:00
  Created wheel for theano-pymc: filename=Theano_PyMC-1.1.2-py3-none-any.whl size=1529959 sha256=dd928902c26f7f646dc705e1d64de205fdb1591ba3c562327f0d2869b8003f5d
  Stored in directory: /root/.cache/pip/wheels/fb/d3/25/6c443075ee21f3b6a59a6a180b55657b012722e1b89f8ccbe8
Successfully built theano-pymc


In [ ]:
import pandas as pd
import scanpy as sc
import numpy as np
from anndata import AnnData

# Standard ML Models for comparison
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

# Splitting data into training/testing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import scale



# Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error

# Distributions
import scipy
import numpy as np

# # PyMC3 for Bayesian Inference
# import pymc3 as pm

In [ ]:
##put together data frames
dfa2=pd.read_csv('/content/drive/MyDrive/The One True Folder/Sandia-Cell-NC State/Ben Supervised Learning Experiments/top10.txt', delimiter = '\t')# Select only relevant variables
#dfa2

#samples that are antibacterial are labeled with either cd14 or b6
cd14=dfa2['sample'].str.contains('CD14|b6')
dfa2['cd14']=cd14


df=dfa2


## Prepare Data
- Uploaded sequencing data frame contains only important genes, as determined in a previous experimental analyis from Dr. Raga Krishnakumar.

In [ ]:
##subset variables
def format_data(df):
    labels = df['cd14']
    df2=df.drop(columns=['sample'])
    X_train, X_test, y_train, y_test = train_test_split(df2, labels,
                                                        test_size = 0.25,
                                                        random_state=42)
    return X_train, X_test, y_train, y_test

In [ ]:

X_train, X_test, y_train, y_test = format_data(df)

scaler = sklearn.preprocessing.StandardScaler().fit(X_train)
X_test.head()

,Ccl20,Ccl5,Cxcl10,Ly6c1,Ly6d,Ly6g,Plac8,Ppbp,Rsad2,Saa3,cd14
9757,0.0,0.099915,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.831545,True
8538,0.0,0.000000,0.034556,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,True
1595,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,False
3333,0.0,0.000000,0.000000,0.110539,0.000000,0.0,0.0,0.0,0.0,0.000000,False
3852,0.0,0.000000,0.000000,0.031530,0.097009,0.0,0.0,0.0,0.0,0.000000,False


In [ ]:
X_train2=pd.DataFrame(scaler.transform(X_train))
X_train2.columns=X_train.columns
X_test2=pd.DataFrame(scaler.transform(X_test))
X_test2.columns=X_test.columns


In [ ]:
def evaluate_predictions(predictions, true):
    mae = np.mean(abs(predictions - true))
    rmse = np.sqrt(np.mean((predictions - true) ** 2))

    return mae, rmse

In [ ]:
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix


## Create evaluation Method

In [ ]:
# Evaluate several ml models by training on training set and testing on testing set
##TO DO - none of these models have been optimized, so that's on the to-do list
def evaluate(X_train, X_test, y_train, y_test):
    # Names of models
    model_name_list = ['Logistic Regression', 'SVM','Random Forest', 'NN', 'GaussianNB', 'SVM-rbf']
    X_train = X_train.drop(columns=['cd14'])
    X_test = X_test.drop(columns=['cd14'])

    # Instantiate the models
    model1 = LogisticRegression(multi_class='ovr', random_state=0, max_iter=5000)
    model2 = svm.LinearSVC(random_state=0, max_iter=5000)
    model3 = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
    model4 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1, max_iter=5000)
    model5 = GaussianNB()
    model6 = svm.SVC(random_state=0, max_iter=5000, kernel='rbf')

    # Dataframe for results
    results = pd.DataFrame(columns=['accuracy', 'precision','recall','specificity','f1'], index = model_name_list)
    # Dataframe for predictions
    preds = {}
    impGenes = {}
    # Train and predict with each model
    for i, model in enumerate([model1, model2, model3, model4, model5, model6]):
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)

        accuracy = accuracy_score(y_test, predictions)
        #print('Accuracy: %f' % accuracy)
        # precision tp / (tp + fp)
        precision = precision_score(y_test, predictions)
        #print('Precision: %f' % precision)
        # recall: tp / (tp + fn)
        recall = recall_score(y_test, predictions)
        #print('Recall: %f' % recall)
        # f1: 2 tp / (2 tp + fp + fn)
        f1 = f1_score(y_test, predictions)
        #print('F1 score: %f' % f1)
        # Insert results into the dataframe
        tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
        specificity = tn / (tn+fp)
        #tn / (tn + fp)
        model_name = model_name_list[i]
        results.loc[model_name, :] = [accuracy, precision, recall, specificity, f1]
        preds[model_name_list[i]]=predictions
        if model_name=='Random Forest':
            importance = model.feature_importances_
            impGenes[model_name_list[i]]=importance
        elif model_name=='Logistic Regression':
            importance = model.coef_[0]
            impGenes[model_name_list[i]]=importance
        else:
            impGenes[model_name_list[i]]=[]
    # Median Value Baseline Metrics
    #baseline = np.median(y_train)
    #baseline_mae = np.mean(abs(baseline - y_test))
    #baseline_rmse = np.sqrt(np.mean((baseline - y_test) ** 2))

    #results.loc['Baseline', :] = [baseline_mae, baseline_rmse]

    return results,preds,impGenes

In [ ]:
results,predictions,impGenes= evaluate(X_train, X_test, y_train, y_test)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
results2,predictions2,impGenes2= evaluate(X_train2, X_test2, y_train, y_test)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
results

,accuracy,precision,recall,specificity,f1
Logistic Regression,0.639839,0.601028,0.907609,0.351713,0.723167
SVM,0.640644,0.600611,0.915373,0.345029,0.725315
Random Forest,0.655533,0.613565,0.906056,0.385965,0.731661
NN,0.51831,0.51831,1.0,0.0,0.682746
GaussianNB,0.612072,0.575349,0.960404,0.23726,0.719604
SVM-rbf,0.641851,0.601324,0.916925,0.345865,0.726322


In [ ]:
results2

,accuracy,precision,recall,specificity,f1
Logistic Regression,0.641851,0.60343,0.901398,0.362573,0.722914
SVM,0.641449,0.601224,0.915373,0.3467,0.725762
Random Forest,0.655533,0.613565,0.906056,0.385965,0.731661
NN,0.686117,0.65679,0.826087,0.535505,0.731774
GaussianNB,0.612072,0.575349,0.960404,0.23726,0.719604
SVM-rbf,0.647887,0.610251,0.887422,0.390142,0.723189


- Accuracies are much lower that previously reported with this set of genes.
- Are these truly the best set of genes to use with the classifiers?
- I'm going to try do derive a set of important genes myself, and then retry classical ML methods on this.

In [ ]:
import pandas as pd
import scanpy as sc
import numpy as np
from anndata import AnnData


file_path2 = '/content/drive/My Drive/The One True Folder/Sandia-Cell-NC State/final_combat_data.txt'

anndata1 = sc.read_csv(file_path2).T

- For top 10 genes, Caitlin compared differentially expressed genes from findmarkers and from different sides of the UMAP.
- She said there was not much difference between the two.
- She ultimately used genes from different sides of the UMAP.
- I'm going to try to use the scanpy equivalent of findmarkers;rank_genes_groups

## Preparing the raw data

In [ ]:
def assign_label(cell_name):
  if 'b6' in cell_name:
    return "Antimicrobial"
  elif 'bcCD14' in cell_name:
    return 'Antimicrobial'
  elif 'bcScr' in cell_name:
    return 'Suboptimal'
  else:
    return 'unknown'

anndata1.obs['cell_type'] = anndata1.obs.index.map(assign_label)

In [ ]:
#Normalize total counts to 10,000 per cell
sc.pp.normalize_total(anndata1, target_sum=1e4)

#Log-transform the data
sc.pp.log1p(anndata1)

In [ ]:
sc.tl.rank_genes_groups(anndata1, groupby="cell_type", method="wilcoxon")

## Find differentially expressed genes the groups. Use logfold changes.

In [ ]:
=

ranked_genes_sub = sc.get.rank_genes_groups_df(anndata1, group="Suboptimal",pval_cutoff=.05, log2fc_min=1)
top_10_sub=ranked_genes_sub.sort_values('logfoldchanges',ascending=False).head(10)


In [ ]:
# adding all the genes to alist

top_20_list = []
for name in top_10_sub.names:
  top_20_list.append(name)

for name in top_10_anti.names:
  top_20_list.append(name)
print(top_20_list)

['Ly6d', 'Tchh', 'Ly6g', 'Ly6c2', 'Csf2', 'Flt4', 'Sele', 'Psca', 'Cxcl2', 'Tgtp1', '3300005D01Rik', 'Nkx2-5', 'Hoxb9', 'Pjvk', 'Cdh13', 'Angptl7', 'Hsf2bp', 'Cd14', 'Prex2', 'Sox5']


- I have the top 10 differentially expressed genes for both groups.
- I think I'm just going to splice these lists together and use these as my genes for the ML models.
- This is slightly different from how Caitlin did it, but hopefully it should work.

In [ ]:
print(anndata1.obs["cell_type"])

b6_AAACGCTGTAATGCGG.1               Antimicrobial
b6_AAAGAACTCAACTGGT.1               Antimicrobial
b6_AAAGAACTCTCATTAC.1               Antimicrobial
b6_AAAGGTACAGAACGCA.1               Antimicrobial
b6_AAAGTCCAGACATAAC.1               Antimicrobial
                                        ...      
bcCD14_18hLPS_TTTAGTCCAAACTCTG.1    Antimicrobial
bcCD14_18hLPS_TTTATGCTCTTGCAGA.1    Antimicrobial
bcCD14_18hLPS_TTTCACATCCAGCCTT.1    Antimicrobial
bcCD14_18hLPS_TTTCCTCGTCTGCATA.1    Antimicrobial
bcCD14_18hLPS_TTTGTTGTCAGGAGAC.1    Antimicrobial
Name: cell_type, Length: 9938, dtype: category
Categories (2, object): ['Antimicrobial', 'Suboptimal']


## Keep only the top 20 genes as previously determined

In [ ]:
## Create data frame object containing filtering for the top 20 genes only

df_a = anndata1.to_df()
df_filter=df_a[top_20_list]
df_filter['cell_type'] = (anndata1.obs['cell_type'] == "Antimicrobial").astype(int)
print(df_filter.head())



                       Ly6d  Tchh  Ly6g  Ly6c2  Csf2  Flt4  Sele  Psca  Cxcl2  \
b6_AAACGCTGTAATGCGG.1   0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0    0.0   
b6_AAAGAACTCAACTGGT.1   0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0    0.0   
b6_AAAGAACTCTCATTAC.1   0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0    0.0   
b6_AAAGGTACAGAACGCA.1   0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0    0.0   
b6_AAAGTCCAGACATAAC.1   0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0    0.0   

                       Tgtp1  ...    Nkx2-5     Hoxb9  Pjvk     Cdh13  \
b6_AAACGCTGTAATGCGG.1    0.0  ...  1.111168  0.000000   0.0  0.000000   
b6_AAAGAACTCAACTGGT.1    0.0  ...  0.000000  0.000000   0.0  0.000000   
b6_AAAGAACTCTCATTAC.1    0.0  ...  0.000000  1.272991   0.0  0.496451   
b6_AAAGGTACAGAACGCA.1    0.0  ...  0.000000  0.606629   0.0  0.000000   
b6_AAAGTCCAGACATAAC.1    0.0  ...  0.000000  1.136823   0.0  0.000000   

                        Angptl7  Hsf2bp  Cd14     Prex2      Sox5  cell_ty

<ipython-input-131-771577bf2519>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter['cell_type'] = (anndata1.obs['cell_type'] == "Antimicrobial").astype(int)


## Prepare data for ML

In [ ]:
labels = df_filter['cell_type']
#df_ml=df_filter.drop(columns=['cell_type'])
X_train, X_test, y_train, y_test = train_test_split(df_ml, labels,
                                                    test_size = 0.25,
                                                    random_state=42)

In [ ]:
scaler = sklearn.preprocessing.StandardScaler().fit(X_train)

In [ ]:
X_train2=pd.DataFrame(scaler.transform(X_train))
X_train2.columns=X_train.columns
X_test2=pd.DataFrame(scaler.transform(X_test))
X_test2.columns=X_test.columns

In [ ]:
# Evaluate several ml models by training on training set and testing on testing set
##TO DO - none of these models have been optimized, so that's on the to-do list
def evaluate2(X_train, X_test, y_train, y_test):
    # Names of models
    model_name_list = ['Logistic Regression', 'SVM','Random Forest', 'NN', 'GaussianNB', 'SVM-rbf']

    # Instantiate the models
    model1 = LogisticRegression(multi_class='ovr', random_state=0, max_iter=5000)
    model2 = svm.LinearSVC(random_state=0, max_iter=5000)
    model3 = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
    model4 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1, max_iter=5000)
    model5 = GaussianNB()
    model6 = svm.SVC(random_state=0, max_iter=5000, kernel='rbf')

    # Dataframe for results
    results = pd.DataFrame(columns=['accuracy', 'precision','recall','specificity','f1'], index = model_name_list)
    # Dataframe for predictions
    preds = {}
    impGenes = {}
    # Train and predict with each model
    for i, model in enumerate([model1, model2, model3, model4, model5, model6]):
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)

        accuracy = accuracy_score(y_test, predictions)
        #print('Accuracy: %f' % accuracy)
        # precision tp / (tp + fp)
        precision = precision_score(y_test, predictions)
        #print('Precision: %f' % precision)
        # recall: tp / (tp + fn)
        recall = recall_score(y_test, predictions)
        #print('Recall: %f' % recall)
        # f1: 2 tp / (2 tp + fp + fn)
        f1 = f1_score(y_test, predictions)
        #print('F1 score: %f' % f1)
        # Insert results into the dataframe
        tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
        specificity = tn / (tn+fp)
        #tn / (tn + fp)
        model_name = model_name_list[i]
        results.loc[model_name, :] = [accuracy, precision, recall, specificity, f1]
        preds[model_name_list[i]]=predictions
        if model_name=='Random Forest':
            importance = model.feature_importances_
            impGenes[model_name_list[i]]=importance
        elif model_name=='Logistic Regression':
            importance = model.coef_[0]
            impGenes[model_name_list[i]]=importance
        else:
            impGenes[model_name_list[i]]=[]
    # Median Value Baseline Metrics
    #baseline = np.median(y_train)
    #baseline_mae = np.mean(abs(baseline - y_test))
    #baseline_rmse = np.sqrt(np.mean((baseline - y_test) ** 2))

    #results.loc['Baseline', :] = [baseline_mae, baseline_rmse]

    return results,preds,impGenes

In [ ]:
results2,predictions2,impGenes2= evaluate2(X_train2, X_test2, y_train, y_test)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
results2

,accuracy,precision,recall,specificity,f1
Logistic Regression,0.927968,0.957886,0.900621,0.957393,0.928371
SVM,0.92837,0.958678,0.900621,0.958229,0.928743
Random Forest,0.894165,0.949956,0.840062,0.952381,0.891636
NN,0.929577,0.95878,0.90295,0.958229,0.930028
GaussianNB,0.687726,0.962094,0.41382,0.982456,0.578719
SVM-rbf,0.927565,0.951876,0.906056,0.95071,0.928401


- Awesome, we got much higher accuracy when we used my set of genes. This is good!! We are able to replicate her results.
- It is unclear why the set of important genes provided to us did not work very well as features for ML models, but deriving a set of important genes myself seemed to do well with these various models on first pass, with no finetuning.